# Getting started with DeepMatcher

Note: you can run **[this notebook live in Google Colab](https://colab.research.google.com/github/anhaidgroup/deepmatcher/blob/master/examples/getting_started.ipynb)** and use free GPUs provided by Google.

This tutorial describes how to effortlessly perform entity matching using deep neural networks. Specifically, we will see how to match pairs of tuples (also called data records or table rows) to determine if they refer to the same real world entity. To do so, we will need labeled examples as input, i.e., tuple pairs which have been annotated as matches or non-matches. This will be used to train our neural network using supervised learning. At the end of this tutorial, you will have a trained neural network as output which you can easily apply to unlabeled tuple pairs to make predictions.

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Mounted at /content/drive


As an overview, here are the 4 steps to use `deepmatcher` which we will go through in this tutorial:

<ol start="0">
  <li>Setup</li>
  <li>Process labeled data</li>
  <li>Define neural network model</li>
  <li>Train model</li>
  <li>Apply model to new data</li>
</ol>

Let's begin!

## Step 0. Setup

If you are running this notebook inside Colab, you will first need to install necessary packages by running the code below:

In [ ]:
try:
    import deepmatcher
except:
    !pip install -qqq deepmatcher

In [ ]:
# contorna erro legacy
!pip install torchtext==0.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Now let's import `deepmatcher` which will do all the heavy lifting to build and train neural network models for entity matching. 

In [ ]:
import pandas as pd
import deepmatcher as dm

We recommend having a GPU available for the training in Step 4. In case a GPU is not available, we will use all available CPU cores. You can run the following command to determine if a GPU is available and will be used for training:

In [ ]:
import torch
torch.cuda.is_available()

True

### Download sample data for entity matching

Now let's get some sample data to play with in this tutorial. We will need three sets of labeled data and one set of unlabeled data:

1. **Training Data:** This is used for training our neural network model.
2. **Validation Data:** This is used for determining the configuration (i.e., hyperparameters) of our model in such a way that the model does not overfit to the training set.
3. **Test Data:** This is used to estimate the performance of our trained model on unlabeled data.
4. **Unlabeled Data:** The trained model is applied on this data to obtain predictions, which can then be used for downstream tasks in practical application scenarios.

We download these four data sets to the `sample_data` directory:

In [ ]:
#!mkdir -p /content/drive/MyDrive/IC/unirTeste
#!wget -qnc -P /content/drive/MyDrive/IC/unirTeste/ http://pages.cs.wisc.edu/~anhai/data1/deepmatcher_data/Structured.zip

In [ ]:
#!unzip -q /content/drive/MyDrive/IC/unirTeste/Structured.zip -d /content/drive/MyDrive/IC/unirTeste

In [ ]:
#!ls /content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar

In [ ]:
'''
def merge_data(labeled, table_a, table_b, output):
  merged_csv = pd.read_csv(labeled).rename(columns={'ltable_id': 'left_id', 'rtable_id': 'right_id'})
  table_a_csv = pd.read_csv(table_a)
  table_a_csv = table_a_csv.rename(columns={col: 'left_' + col for col in table_a_csv.columns})
  table_b_csv = pd.read_csv(table_b)
  table_b_csv = table_b_csv.rename(columns={col: 'right_' + col for col in table_b_csv.columns})
  merged_csv = pd.merge(merged_csv, table_a_csv, on='left_id')
  merged_csv = pd.merge(merged_csv, table_b_csv, on='right_id')
  merged_csv['id'] = merged_csv[['left_id', 'right_id']].apply(lambda row: '_'.join([str(c) for c in row]), axis=1)
  del merged_csv['left_id']
  del merged_csv['right_id']
  merged_csv.to_csv(output, index=False)
'''

"\ndef merge_data(labeled, table_a, table_b, output):\n  merged_csv = pd.read_csv(labeled).rename(columns={'ltable_id': 'left_id', 'rtable_id': 'right_id'})\n  table_a_csv = pd.read_csv(table_a)\n  table_a_csv = table_a_csv.rename(columns={col: 'left_' + col for col in table_a_csv.columns})\n  table_b_csv = pd.read_csv(table_b)\n  table_b_csv = table_b_csv.rename(columns={col: 'right_' + col for col in table_b_csv.columns})\n  merged_csv = pd.merge(merged_csv, table_a_csv, on='left_id')\n  merged_csv = pd.merge(merged_csv, table_b_csv, on='right_id')\n  merged_csv['id'] = merged_csv[['left_id', 'right_id']].apply(lambda row: '_'.join([str(c) for c in row]), axis=1)\n  del merged_csv['left_id']\n  del merged_csv['right_id']\n  merged_csv.to_csv(output, index=False)\n"

In [ ]:
'''
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/train.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_train.csv')
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/valid.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_valid.csv')
merge_data(
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/test.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', 
    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_test.csv')
'''

"\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/train.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_train.csv')\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/valid.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/joined_valid.csv')\nmerge_data(\n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/test.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableA.csv', \n    '/content/drive/MyDrive/IC/unirTeste/Structured/DBLP-GoogleScholar/tableB.csv', \n    '/

To get an idea of how our data looks like, let's take a peek at the training dataset:

In [ ]:
#pd.read_csv('/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Structured/DBLP-GoogleScholar/joined_train.csv').head()

## Step 1. Process labeled data

Before we can use our data for training, `deepmatcher` needs to first load and process it in order to prepare it for neural network training. Currently `deepmatcher` only supports processing CSV files. Each CSV file is assumed to have the following kinds of columns:

* **"Left" attributes (required):** Our goal is to match tuple pairs. "Left" attributes are columns that correspond to the "left" tuple or the first tuple in the tuple pair. These column names are expected to be prefixed with "left_" by default.
* **"Right" attributes (required):** "Right" attributes are columns that correspond to the "right" tuple or the second tuple in the tuple pair. These column names are expected to be prefixed with "right_" by default.
* **Label column (required for train, validation, test):** Column containing the labels (match or non-match) for each tuple pair. Expected to be named "label" by default
* **ID column (required):** Column containing a unique ID for each tuple pair. This is for evaluation convenience.  Expected to be named "id" by default.

More details on what data processing involves and ways to customize it are described in **[this notebook](https://nbviewer.jupyter.org/github/anhaidgroup/deepmatcher/blob/master/examples/data_processing.ipynb)**. 

### Processing labeled data
In order to process our train, validation and test CSV files we call `dm.data.process` in the following code snippet which will load and process the CSV files and return three processed `MatchingDataset` objects respectively. These dataset objects will later be used for training and evaluation. The basic parameters to `dm.data.process` are as follows:

* **path (required): ** The path where all data is stored. This includes train, validation and test. `deepmatcher` may create new files in this directory to store information about these data sets. This allows subsequent `dm.data.process` calls to be much faster.
* **train (required): ** File name of training data in `path` directory.
* **validation (required): ** File name of validation data in `path` directory.
* **test (optional): ** File name of test data in `path` directory.
* **ignore_columns (optional): ** Any columns in the CSV files that you may want to ignore for the purposes of training. These should be included here. 

Note that the train, validation and test CSVs must all share the same schema, i.e., they should have the same columns. Processing data involves several steps and can take several minutes to complete, especially if this is the first time you are running the `deepmatcher` package.

NOTE: If you are running this in Colab, you may get a message saying 'Memory usage is close to the limit.' You can safely ignore it for now. We are working on reducing the memory footprint.

In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip --directory-prefix=/root/.vector_cache
!unzip /root/.vector_cache/wiki.en.zip -d /root/.vector_cache/
!rm /root/.vector_cache/wiki.en.vec

--2022-08-09 22:01:26--  https://dl.fbaipublicfiles.com/fasttext/vectors-wiki/wiki.en.zip
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 172.67.9.4, 104.22.74.142, 104.22.75.142, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|172.67.9.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10356881291 (9.6G) [application/zip]
Saving to: ‘/root/.vector_cache/wiki.en.zip’

wiki.en.zip         100%[===================>]   9.65G  29.8MB/s    in 5m 23s  

2022-08-09 22:06:50 (30.5 MB/s) - ‘/root/.vector_cache/wiki.en.zip’ saved [10356881291/10356881291]

Archive:  /root/.vector_cache/wiki.en.zip
  inflating: /root/.vector_cache/wiki.en.vec  
  inflating: /root/.vector_cache/wiki.en.bin  


In [ ]:
'''train, validation, test = dm.data.process(
    path='/content/drive/MyDrive/IC/ICReteste/Deepmatcher/TesteErros/TesteErrosLabel/Erros6_10/Datasets/Dirty/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test_error.csv')'''

#### Peeking at processed data
Let's take a look at how the processed data looks like. To do this, we get the raw `pandas` table corresponding to the processed training dataset object. 

In [ ]:
'''train_table = train.get_raw_table()
train_table.head()'''

,label,left_title,left_authors,left_venue,left_year,right_title,right_authors,right_venue,right_year,id
0,0,the demarcation protocol : a technique for mai...,"d barbarс , h garcia-molina",,,local verification of global integrity constra...,,,,2563_6425
1,0,temporal conditions and integrity constraints ...,"a sistla , o wolfson",,1995.0,local verification of global integrity constra...,,,,1610_6425
2,0,the role of integrity constraints in database ...,"m vermeer , p apers",vldb,,local verification of global integrity constra...,,,,1999_6425
3,1,the demarcation protocol : a technique for mai...,"d barbarс , h garcia-molina",,,protocol : a technique for maintaining linear ...,", h garcia-molina",,,2563_44470
4,0,the demarcation protocol : a technique for mai...,"d barbarс , h garcia-molina",,,a locking protocol for resource coordination i...,"da menasce , gj popek , rr muntz",,1980.0,2563_55077


The processed attribute values have been tokenized and lowercased so they may not look exactly the same as the input training data. These modifications help the neural network generalize better, i.e., perform better on data not trained on. 

## Step 2. Define neural network model

In this step you tell `deepmatcher` what kind of neural network you would like to use for entity matching. The easiest way to do this is to use one of the several kinds of neural network models that comes built-in with `deepmatcher`. To use a built-in network, construct a `dm.MatchingModel` as follows:

`model = dm.MatchingModel(attr_summarizer='<TYPE>')`

where `<TYPE>` is one of `sif`, `rnn`, `attention` or `hybrid`. If you are not familiar with what these mean, we strongly recommend taking a look at either **[slides from our talk on deepmatcher](http://bit.do/deepmatcher-talk)** for a high level overview, or **[our paper](http://pages.cs.wisc.edu/~anhai/papers1/deepmatcher-sigmod18.pdf)** for a more detailed explanation. Here we give briefly describe the intuition behind these four model types:
* **sif:** This model considers the **words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **rnn:** This model considers the **sequences of words** present in each attribute value pair to determine a match or non-match.
* **attention:** This model considers the **alignment of words** present in each attribute value pair to determine a match or non-match. It does not take word order into account.
* **hybrid:** This model considers the **alignment of sequences of words** present in each attribute value pair to determine a match or non-match. This is the default.

`deepmatcher` is highly customizable and allows you to tune almost every aspect of the neural network model for your application scenario. **[This tutorial](https://nbviewer.jupyter.org/github/anhaidgroup/deepmatcher/blob/master/examples/matching_models.ipynb)** discusses the structure of `MatchingModel`s and how they can be customized.

For this tutorial, let's create a `hybrid` model for entity matching:

In [ ]:
'''model = dm.MatchingModel(attr_summarizer='attention')'''

## Step 3. Train model

Next, we train the defined neural network model using the processed training and validation data. To do so, we call the `run_train` method which takes the following basic parameters:

* **train:** The processed training dataset object (of type `MatchingDataset`).
* **validation:** The processed validation dataset object (of type `MatchingDataset`).
* **epochs:** Number of times to go over the entire `train` data for training the model.
* **batch_size:** Number of labeled examples (tuple pairs) to use for each training step. This value may be increased if you have a lot of training data and would like to speed up training. The optimal value is dataset dependent.
* **best_save_path:** Path to save the best model.
* **pos_neg_ratio**: The ratio of the weight of positive examples (matches) to weight of negative examples (non-matches). This value should be increased if you have fewer matches than non-matches in your data. The optimal value is dataset dependent.

Many other aspects of the training algorithm can be customized. For details on this, please refer the API documentation for **[run_train]()**

In [ ]:
'''model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)'''

## Novos Testes

In [ ]:
!git clone https://github.com/pauloh48/IC.git

Cloning into 'IC'...
remote: Enumerating objects: 985, done.
remote: Counting objects: 100% (55/55), done.
remote: Compressing objects: 100% (51/51), done.
remote: Total 985 (delta 16), reused 7 (delta 0), pack-reused 930
Receiving objects: 100% (985/985), 252.10 MiB | 28.31 MiB/s, done.
Resolving deltas: 100% (600/600), done.
Checking out files: 100% (1274/1274), done.


### RNN

#### STRUCTURED

##### Amazon-Google

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 1762802
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    5.9 | Load Time:    4.9 || F1:  23.69 | Prec:  38.22 | Rec:  17.17 || Ex/s: 640.83

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 1 || Run Time:    0.9 | Load Time:    1.5 || F1:  46.73 | Prec:  51.55 | Rec:  42.74 || Ex/s: 939.09

* Best F1: tensor(46.7290, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    5.8 | Load Time:    4.8 || F1:  99.15 | Prec:  98.59 | Rec:  99.71 || Ex/s: 647.39

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.9 | Load Time:    1.5 || F1:  51.11 | Prec:  60.12 | Rec:  44.44 || Ex/s: 954.60

---------------------

Loading best model...
Training done.


tensor(57.6613, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 7


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 7 || Run Time:    0.9 | Load Time:    1.6 || F1:  53.85 | Prec:  51.15 | Rec:  56.84 || Ex/s: 941.74



tensor(53.8462, device='cuda:0')

##### Beer

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.3 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 411.43

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.1 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 571.43

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.3 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 442.86

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.0 | Load Time:    0.1 || F1:  50.00 | Prec:  34.21 | Rec:  92.86 || Ex/s: 611.34

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.3 | Load Time:    0.3 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 441.36

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.0 | Load Time:    0.1 || F1:  50.00 | Prec:  34.21 | Rec:  92.86 || Ex/s: 604.63

---------------------

Loading best model...
Training done.


tensor(57.7778, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 5 || Run Time:    0.0 | Load Time:    0.1 || F1:  52.17 | Prec:  37.50 | Rec:  85.71 || Ex/s: 604.89



tensor(52.1739, device='cuda:0')

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    9.1 | Load Time:   12.5 || F1:  90.62 | Prec:  88.20 | Rec:  93.17 || Ex/s: 343.25

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    1.4 | Load Time:    3.9 || F1:  94.75 | Prec:  90.39 | Rec:  99.55 || Ex/s: 468.68

* Best F1: tensor(94.7481, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:21


Finished Epoch 15 || Run Time:    9.4 | Load Time:   12.6 || F1:  99.66 | Prec:  99.33 | Rec: 100.00 || Ex/s: 338.15

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    1.5 | Load Time:    4.1 || F1:  97.89 | Prec:  96.70 | Rec:  99.10 || Ex/s: 440.90

---------------------

Loading best model...
Training done.


tensor(98.6547, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 5 || Run Time:    1.4 | Load Time:    3.9 || F1:  97.77 | Prec:  96.70 | Rec:  98.87 || Ex/s: 462.18



tensor(97.7728, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   20.1 | Load Time:   25.5 || F1:  81.06 | Prec:  75.78 | Rec:  87.12 || Ex/s: 377.53

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    3.1 | Load Time:    7.8 || F1:  89.67 | Prec:  91.37 | Rec:  88.04 || Ex/s: 524.69

* Best F1: tensor(89.6716, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:44


Finished Epoch 15 || Run Time:   20.0 | Load Time:   25.3 || F1:  99.75 | Prec:  99.69 | Rec:  99.81 || Ex/s: 379.75

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    3.1 | Load Time:    7.8 || F1:  92.58 | Prec:  92.45 | Rec:  92.71 || Ex/s: 526.83

---------------------

Loading best model...
Training done.


tensor(92.6670, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 13


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 13 || Run Time:    3.1 | Load Time:    8.0 || F1:  92.06 | Prec:  91.43 | Rec:  92.71 || Ex/s: 514.48



tensor(92.0650, device='cuda:0')

##### Walmart-Amazon

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2576402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    8.1 | Load Time:    7.1 || F1:  31.75 | Prec:  60.49 | Rec:  21.53 || Ex/s: 405.73

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    1.3 | Load Time:    2.2 || F1:  59.15 | Prec:  71.85 | Rec:  50.26 || Ex/s: 590.07

* Best F1: tensor(59.1463, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 15 || Run Time:    8.0 | Load Time:    6.9 || F1: 100.00 | Prec: 100.00 | Rec: 100.00 || Ex/s: 410.46

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    1.3 | Load Time:    2.2 || F1:  64.07 | Prec:  75.89 | Rec:  55.44 || Ex/s: 590.94

---------------------

Loading best model...
Training done.


tensor(65.9155, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 9 || Run Time:    1.2 | Load Time:    2.2 || F1:  67.62 | Prec:  75.64 | Rec:  61.14 || Ex/s: 603.09



tensor(67.6218, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 1762802
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    6.7 | Load Time:   11.1 || F1:  17.96 | Prec:  28.32 | Rec:  13.15 || Ex/s: 321.32

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    0.9 | Load Time:    3.3 || F1:   0.97 | Prec: 100.00 | Rec:   0.49 || Ex/s: 451.29

* Best F1: tensor(0.9662, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 15 || Run Time:    6.6 | Load Time:   11.0 || F1:  99.19 | Prec:  98.56 | Rec:  99.84 || Ex/s: 325.12

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    1.0 | Load Time:    3.3 || F1:  31.35 | Prec:  44.25 | Rec:  24.27 || Ex/s: 448.91

---------------------

Loading best model...
Training done.


tensor(40.3628, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 5 || Run Time:    0.9 | Load Time:    3.3 || F1:  40.55 | Prec:  38.60 | Rec:  42.72 || Ex/s: 452.45



tensor(40.5530, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    9.7 | Load Time:   15.0 || F1:  71.18 | Prec:  64.34 | Rec:  79.65 || Ex/s: 299.60

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    1.4 | Load Time:    4.7 || F1:  79.92 | Prec:  68.72 | Rec:  95.50 || Ex/s: 403.70

* Best F1: tensor(79.9246, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:24


Finished Epoch 15 || Run Time:    9.6 | Load Time:   15.0 || F1:  99.81 | Prec:  99.85 | Rec:  99.77 || Ex/s: 301.07

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    1.4 | Load Time:    4.7 || F1:  92.43 | Prec:  91.41 | Rec:  93.47 || Ex/s: 406.98

---------------------

Loading best model...
Training done.


tensor(92.8090, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 10


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 10 || Run Time:    1.4 | Load Time:    4.6 || F1:  92.22 | Prec:  91.01 | Rec:  93.47 || Ex/s: 411.31



tensor(92.2222, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='rnn')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='rnn_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 2169602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   21.0 | Load Time:   30.1 || F1:  67.41 | Prec:  60.54 | Rec:  76.05 || Ex/s: 336.51

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


Finished Epoch 1 || Run Time:    3.1 | Load Time:    9.2 || F1:  82.35 | Prec:  78.71 | Rec:  86.36 || Ex/s: 465.96

* Best F1: tensor(82.3529, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:51


Finished Epoch 15 || Run Time:   21.4 | Load Time:   30.2 || F1:  99.80 | Prec:  99.91 | Rec:  99.69 || Ex/s: 333.63

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:12


Finished Epoch 15 || Run Time:    3.2 | Load Time:    9.2 || F1:  87.75 | Prec:  87.47 | Rec:  88.04 || Ex/s: 461.74

---------------------

Loading best model...
Training done.


tensor(87.8366, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-f

Finished Epoch 11 || Run Time:    3.2 | Load Time:    9.4 || F1:  87.02 | Prec:  88.58 | Rec:  85.51 || Ex/s: 454.83



tensor(87.0185, device='cuda:0')

### SIF

#### STRUCTURED

##### Amazon-Google

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 1 || Run Time:    3.3 | Load Time:    5.0 || F1:  12.53 | Prec:  27.01 | Rec:   8.15 || Ex/s: 825.57

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 1 || Run Time:    0.6 | Load Time:    1.5 || F1:  22.80 | Prec:  47.95 | Rec:  14.96 || Ex/s: 1075.86

* Best F1: tensor(22.8013, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 15 || Run Time:    3.0 | Load Time:    4.8 || F1:  90.71 | Prec:  86.21 | Rec:  95.71 || Ex/s: 884.81

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.6 | Load Time:    1.5 || F1:  43.85 | Prec:  46.01 | Rec:  41.88 || Ex/s: 1084.92

---------------------

Loading best model...
Training done.


tensor(46.9565, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 6 || Run Time:    0.6 | Load Time:    1.5 || F1:  44.76 | Prec:  49.23 | Rec:  41.03 || Ex/s: 1077.09



tensor(44.7552, device='cuda:0')

##### Beer

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.3 || F1:  16.90 | Prec:  19.35 | Rec:  15.00 || Ex/s: 542.57

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 632.57

* Best F1: tensor(0., device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.2 | Load Time:    0.3 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 567.98

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.0 | Load Time:    0.1 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 678.03

---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.2 | Load Time:    0.3 || F1:  18.18 | Prec: 100.00 | Rec:  10.00 || Ex/s: 604.59

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.0 | Load Time:    0.1 || F1:  23.53 | Prec:  66.67 | Rec:  14.29 || Ex/s: 671.41

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.2 | Load Time:    0.3 || F1:  18.18 | Prec: 100.00 | Rec:  10.00 || Ex/s: 563.20

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.0 | Load Time:    0.1 || F1:  23.53 | Prec:  66.67 | Rec:  14.29 || Ex/s: 681.32

---------------------

Loading best model...
Training done.


tensor(23.5294, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 10


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


Finished Epoch 10 || Run Time:    0.0 | Load Time:    0.1 || F1:  13.33 | Prec: 100.00 | Rec:   7.14 || Ex/s: 652.58



tensor(13.3333, device='cuda:0')

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:16


Finished Epoch 1 || Run Time:    4.0 | Load Time:   12.4 || F1:  78.08 | Prec:  80.64 | Rec:  75.68 || Ex/s: 450.95

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    0.8 | Load Time:    3.9 || F1:  93.35 | Prec:  89.14 | Rec:  97.97 || Ex/s: 524.05

* Best F1: tensor(93.3476, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:16


Finished Epoch 15 || Run Time:    4.0 | Load Time:   12.4 || F1:  99.51 | Prec:  99.11 | Rec:  99.92 || Ex/s: 454.30

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    0.8 | Load Time:    3.8 || F1:  96.54 | Prec:  95.58 | Rec:  97.52 || Ex/s: 528.88

---------------------

Loading best model...
Training done.


tensor(96.6368, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 6 || Run Time:    0.9 | Load Time:    3.8 || F1:  96.11 | Prec:  94.95 | Rec:  97.30 || Ex/s: 527.40



tensor(96.1068, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:34


Finished Epoch 1 || Run Time:    9.1 | Load Time:   25.3 || F1:  67.86 | Prec:  64.49 | Rec:  71.59 || Ex/s: 500.27

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 1 || Run Time:    1.9 | Load Time:    7.8 || F1:  83.15 | Prec:  81.94 | Rec:  84.39 || Ex/s: 591.99

* Best F1: tensor(83.1492, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:34


Finished Epoch 15 || Run Time:    9.1 | Load Time:   25.3 || F1:  99.05 | Prec:  98.64 | Rec:  99.47 || Ex/s: 500.68

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 15 || Run Time:    1.9 | Load Time:    7.7 || F1:  85.32 | Prec:  83.02 | Rec:  87.76 || Ex/s: 592.57

---------------------

Loading best model...
Training done.


tensor(85.3953, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 4 || Run Time:    2.2 | Load Time:    7.8 || F1:  84.03 | Prec:  81.24 | Rec:  87.01 || Ex/s: 574.13



tensor(84.0253, device='cuda:0')

##### Walmart-Amazon

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 1 || Run Time:    4.1 | Load Time:    6.9 || F1:  26.01 | Prec:  51.81 | Rec:  17.36 || Ex/s: 560.02

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    0.8 | Load Time:    2.2 || F1:  51.40 | Prec:  46.81 | Rec:  56.99 || Ex/s: 674.71

* Best F1: tensor(51.4019, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:10


Finished Epoch 15 || Run Time:    3.9 | Load Time:    6.9 || F1:  99.91 | Prec:  99.83 | Rec: 100.00 || Ex/s: 566.03

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


Finished Epoch 15 || Run Time:    0.8 | Load Time:    2.2 || F1:  60.23 | Prec:  66.67 | Rec:  54.92 || Ex/s: 681.71

---------------------

Loading best model...
Training done.


tensor(61.7647, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 9 || Run Time:    0.9 | Load Time:    2.2 || F1:  54.49 | Prec:  67.69 | Rec:  45.60 || Ex/s: 667.36



tensor(54.4892, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


Finished Epoch 1 || Run Time:    2.6 | Load Time:   11.0 || F1:   0.00 | Prec:   0.00 | Rec:   0.00 || Ex/s: 420.94

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    0.5 | Load Time:    3.3 || F1:   0.97 | Prec: 100.00 | Rec:   0.49 || Ex/s: 505.33

* Best F1: tensor(0.9662, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:13


Finished Epoch 15 || Run Time:    2.6 | Load Time:   11.0 || F1:  95.54 | Prec:  92.28 | Rec:  99.03 || Ex/s: 422.99

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    0.5 | Load Time:    3.3 || F1:  24.32 | Prec:  32.52 | Rec:  19.42 || Ex/s: 505.14

---------------------

Loading best model...
Training done.


tensor(26.6304, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 9 || Run Time:    0.7 | Load Time:    3.3 || F1:  24.86 | Prec:  28.85 | Rec:  21.84 || Ex/s: 474.41



tensor(24.8619, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


Finished Epoch 1 || Run Time:    4.1 | Load Time:   15.3 || F1:  48.12 | Prec:  59.51 | Rec:  40.39 || Ex/s: 382.21

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    0.9 | Load Time:    4.8 || F1:  69.57 | Prec:  68.87 | Rec:  70.27 || Ex/s: 430.60

* Best F1: tensor(69.5652, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:18


Finished Epoch 15 || Run Time:    4.0 | Load Time:   14.9 || F1:  98.15 | Prec:  96.93 | Rec:  99.40 || Ex/s: 390.84

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    0.8 | Load Time:    4.7 || F1:  77.07 | Prec:  81.09 | Rec:  73.42 || Ex/s: 446.75

---------------------

Loading best model...
Training done.


tensor(77.9191, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 10


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 10 || Run Time:    0.8 | Load Time:    4.6 || F1:  78.97 | Prec:  82.04 | Rec:  76.13 || Ex/s: 452.16



tensor(78.9720, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='sif')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='sif_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 542402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:39


Finished Epoch 1 || Run Time:    9.2 | Load Time:   30.3 || F1:  54.68 | Prec:  54.70 | Rec:  54.66 || Ex/s: 435.98

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


Finished Epoch 1 || Run Time:    2.0 | Load Time:    9.2 || F1:  72.03 | Prec:  68.70 | Rec:  75.70 || Ex/s: 513.73

* Best F1: tensor(72.0320, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:38


Finished Epoch 15 || Run Time:    9.1 | Load Time:   29.9 || F1:  98.79 | Prec:  98.69 | Rec:  98.88 || Ex/s: 442.16

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


Finished Epoch 15 || Run Time:    1.9 | Load Time:    9.1 || F1:  76.51 | Prec:  76.01 | Rec:  77.01 || Ex/s: 518.37

---------------------

Loading best model...
Training done.


tensor(76.7963, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:11


Finished Epoch 12 || Run Time:    1.9 | Load Time:    9.2 || F1:  74.83 | Prec:  74.24 | Rec:  75.42 || Ex/s: 514.94



tensor(74.8261, device='cuda:0')

### ATTENTION

#### STRUCTURED

##### Amazon-Google

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 3429602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   11.8 | Load Time:    5.0 || F1:  29.78 | Prec:  35.59 | Rec:  25.61 || Ex/s: 410.10

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 1 || Run Time:    2.2 | Load Time:    1.5 || F1:  36.63 | Prec:  43.53 | Rec:  31.62 || Ex/s: 620.34

* Best F1: tensor(36.6337, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:16


Finished Epoch 15 || Run Time:   11.3 | Load Time:    4.8 || F1:  91.23 | Prec:  87.52 | Rec:  95.28 || Ex/s: 425.11

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:03


Finished Epoch 15 || Run Time:    2.1 | Load Time:    1.5 || F1:  50.49 | Prec:  59.20 | Rec:  44.02 || Ex/s: 631.90

---------------------

Loading best model...
Training done.


tensor(53.7778, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 8 || Run Time:    2.1 | Load Time:    1.5 || F1:  47.77 | Prec:  50.00 | Rec:  45.73 || Ex/s: 625.97



tensor(47.7679, device='cuda:0')

##### Beer

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:    0.9 | Load Time:    0.3 || F1:  27.59 | Prec:  25.53 | Rec:  30.00 || Ex/s: 220.15

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.1 | Load Time:    0.1 || F1:  11.76 | Prec:  33.33 | Rec:   7.14 || Ex/s: 409.26

* Best F1: tensor(11.7647, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 2 || Run Time:    0.6 | Load Time:    0.3 || F1:  49.23 | Prec:  64.00 | Rec:  40.00 || Ex/s: 285.31

===>  EVAL Epoch 2
Finished Epoch 2 || Run Time:    0.1 | Load Time:    0.1 || F1:  37.68 | Prec:  23.64 | Rec:  92.86 || Ex/s: 404.20

* Best F1: tensor(37.6812, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 3


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    0.6 | Load Time:    0.3 || F1:  95.24 | Prec:  90.91 | Rec: 100.00 || Ex/s: 290.55

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.1 | Load Time:    0.1 || F1:  66.67 | Prec:  57.89 | Rec:  78.57 || Ex/s: 423.26

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    0.6 | Load Time:    0.3 || F1:  95.24 | Prec:  90.91 | Rec: 100.00 || Ex/s: 285.34

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.1 | Load Time:    0.1 || F1:  66.67 | Prec:  57.89 | Rec:  78.57 || Ex/s: 411.39

---------------------

Loading best model...
Training done.


tensor(68.5714, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 10


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 10 || Run Time:    0.1 | Load Time:    0.1 || F1:  64.86 | Prec:  52.17 | Rec:  85.71 || Ex/s: 397.36



tensor(64.8649, device='cuda:0')

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "


* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:28


Finished Epoch 1 || Run Time:   16.7 | Load Time:   12.5 || F1:  84.69 | Prec:  79.55 | Rec:  90.54 || Ex/s: 254.62

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:28


Finished Epoch 15 || Run Time:   16.7 | Load Time:   12.4 || F1:  99.40 | Prec:  98.89 | Rec:  99.92 || Ex/s: 254.58

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    3.2 | Load Time:    3.9 || F1:  98.42 | Prec:  98.64 | Rec:  98.20 || Ex/s: 349.13

---------------------

Loading best model...
Training done.


tensor(98.5377, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:    3.3 | Load Time:    3.9 || F1:  98.32 | Prec:  97.56 | Rec:  99.10 || Ex/s: 346.67



tensor(98.3240, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   37.6 | Load Time:   25.5 || F1:  81.78 | Prec:  74.54 | Rec:  90.58 || Ex/s: 272.66

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:15


Finished Epoch 1 || Run Time:    7.2 | Load Time:    7.8 || F1:  90.62 | Prec:  87.54 | Rec:  93.93 || Ex/s: 381.11

* Best F1: tensor(90.6222, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:02


Finished Epoch 15 || Run Time:   37.5 | Load Time:   25.4 || F1:  99.28 | Prec:  99.04 | Rec:  99.53 || Ex/s: 273.66

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:14


Finished Epoch 15 || Run Time:    7.1 | Load Time:    7.8 || F1:  92.02 | Prec:  90.88 | Rec:  93.18 || Ex/s: 385.22

---------------------

Loading best model...
Training done.


tensor(92.4898, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:    7.2 | Load Time:    7.9 || F1:  91.33 | Prec:  88.79 | Rec:  94.02 || Ex/s: 380.78



tensor(91.3300, device='cuda:0')

##### Walmart-Amazon

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 5234402
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   16.0 | Load Time:    6.9 || F1:  17.22 | Prec:  31.78 | Rec:  11.81 || Ex/s: 268.54

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    3.1 | Load Time:    2.2 || F1:  38.52 | Prec:  36.79 | Rec:  40.41 || Ex/s: 385.26

* Best F1: tensor(38.5185, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


Finished Epoch 15 || Run Time:   16.3 | Load Time:    6.9 || F1:  97.93 | Prec:  97.26 | Rec:  98.61 || Ex/s: 264.90

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    3.1 | Load Time:    2.2 || F1:  50.60 | Prec:  46.90 | Rec:  54.92 || Ex/s: 387.19

---------------------

Loading best model...
Training done.


tensor(54.1872, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 3


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 3 || Run Time:    3.1 | Load Time:    2.2 || F1:  52.92 | Prec:  57.23 | Rec:  49.22 || Ex/s: 387.96



tensor(52.9248, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 3429602
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   11.5 | Load Time:   11.2 || F1:  19.08 | Prec:  22.52 | Rec:  16.56 || Ex/s: 253.80

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 1 || Run Time:    2.1 | Load Time:    3.3 || F1:   1.91 | Prec:  66.67 | Rec:   0.97 || Ex/s: 357.62

* Best F1: tensor(1.9139, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:22


Finished Epoch 15 || Run Time:   11.2 | Load Time:   11.0 || F1:  89.98 | Prec:  86.29 | Rec:  93.99 || Ex/s: 257.78

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:05


Finished Epoch 15 || Run Time:    2.0 | Load Time:    3.3 || F1:  50.88 | Prec:  52.88 | Rec:  49.03 || Ex/s: 365.28

---------------------

Loading best model...
Training done.


tensor(54.6275, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 11 || Run Time:    2.1 | Load Time:    3.3 || F1:  50.91 | Prec:  47.86 | Rec:  54.37 || Ex/s: 353.61



tensor(50.9091, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   18.0 | Load Time:   15.3 || F1:  59.39 | Prec:  51.76 | Rec:  69.67 || Ex/s: 222.76

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 1 || Run Time:    3.4 | Load Time:    4.7 || F1:  83.72 | Prec:  91.69 | Rec:  77.03 || Ex/s: 304.79

* Best F1: tensor(83.7209, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:32


Finished Epoch 15 || Run Time:   17.8 | Load Time:   15.2 || F1:  98.29 | Prec:  97.42 | Rec:  99.17 || Ex/s: 224.86

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:07


Finished Epoch 15 || Run Time:    3.4 | Load Time:    4.7 || F1:  93.02 | Prec:  91.50 | Rec:  94.59 || Ex/s: 306.44

---------------------

Loading best model...
Training done.


tensor(93.2314, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 11


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 11 || Run Time:    3.4 | Load Time:    4.6 || F1:  91.92 | Prec:  90.41 | Rec:  93.47 || Ex/s: 307.57



tensor(91.9158, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='attention')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='attention_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 4332002
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   38.6 | Load Time:   30.1 || F1:  72.46 | Prec:  63.75 | Rec:  83.94 || Ex/s: 250.65

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 1 || Run Time:    7.9 | Load Time:    9.6 || F1:  78.19 | Prec:  67.11 | Rec:  93.64 || Ex/s: 328.78

* Best F1: tensor(78.1896, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:08


Finished Epoch 15 || Run Time:   38.9 | Load Time:   30.0 || F1:  97.39 | Prec:  96.40 | Rec:  98.41 || Ex/s: 249.98

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:17


Finished Epoch 15 || Run Time:    7.7 | Load Time:    9.5 || F1:  86.55 | Prec:  84.25 | Rec:  88.97 || Ex/s: 332.99

* Best F1: tensor(86.5454, device='cuda:0')
Saving best model...
Done.
---------------------

Loading best model...
Training done.


tensor(86.5454, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 15


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 15 || Run Time:    7.7 | Load Time:    9.4 || F1:  87.24 | Prec:  84.56 | Rec:  90.09 || Ex/s: 334.47



tensor(87.2398, device='cuda:0')

### HYBRID

#### STRUCTURED

##### Amazon-Google

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Amazon-Google/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/Amazon-Google/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/Amazon-Google/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/Amazon-Google/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 1 || Run Time:   18.8 | Load Time:    4.8 || F1:  29.66 | Prec:  35.24 | Rec:  25.61 || Ex/s: 290.71

===>  EVAL Epoch 1


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 1 || Run Time:    3.2 | Load Time:    1.5 || F1:  37.85 | Prec:  47.13 | Rec:  31.62 || Ex/s: 489.50

* Best F1: tensor(37.8517, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:23


Finished Epoch 15 || Run Time:   18.7 | Load Time:    4.8 || F1:  97.95 | Prec:  96.79 | Rec:  99.14 || Ex/s: 292.03

===>  EVAL Epoch 15


0% [██████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


Finished Epoch 15 || Run Time:    3.1 | Load Time:    1.5 || F1:  57.80 | Prec:  62.38 | Rec:  53.85 || Ex/s: 489.68

---------------------

Loading best model...
Training done.


tensor(61.7021, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 8


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 8 || Run Time:    3.2 | Load Time:    1.5 || F1:  53.50 | Prec:  53.16 | Rec:  53.85 || Ex/s: 481.15



tensor(53.5032, device='cuda:0')

##### Beer

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Beer/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/Beer/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/Beer/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/Beer/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00
Building vocabulary
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [#] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 1 || Run Time:    1.1 | Load Time:    0.3 || F1:  13.16 | Prec:  13.89 | Rec:  12.50 || Ex/s: 192.67

===>  EVAL Epoch 1
Finished Epoch 1 || Run Time:    0.2 | Load Time:    0.1 || F1:  36.11 | Prec:  22.41 | Rec:  92.86 || Ex/s: 330.81

* Best F1: tensor(36.1111, device='cuda:0')
Saving best model...
Done.
---------------------

===>  TRAIN Epoch 2


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


...

0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 14 || Run Time:    1.0 | Load Time:    0.3 || F1:  98.77 | Prec:  97.56 | Rec: 100.00 || Ex/s: 201.05

===>  EVAL Epoch 14
Finished Epoch 14 || Run Time:    0.2 | Load Time:    0.1 || F1:  84.85 | Prec:  73.68 | Rec: 100.00 || Ex/s: 333.07

---------------------

===>  TRAIN Epoch 15


0% [█] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00


Finished Epoch 15 || Run Time:    1.0 | Load Time:    0.3 || F1:  98.77 | Prec:  97.56 | Rec: 100.00 || Ex/s: 200.45

===>  EVAL Epoch 15
Finished Epoch 15 || Run Time:    0.2 | Load Time:    0.1 || F1:  84.85 | Prec:  73.68 | Rec: 100.00 || Ex/s: 330.13

---------------------

Loading best model...
Training done.


tensor(84.8485, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 9


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 9 || Run Time:    0.2 | Load Time:    0.1 || F1:  66.67 | Prec:  54.55 | Rec:  85.71 || Ex/s: 322.32



tensor(66.6667, device='cuda:0')

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-ACM/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-ACM/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-ACM/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [########] 100% | ETA: 00:00:00
Total time elapsed: 00:00:04


In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:39


Finished Epoch 1 || Run Time:   28.6 | Load Time:   11.8 || F1:  88.28 | Prec:  86.47 | Rec:  90.17 || Ex/s: 183.78

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:40


Finished Epoch 15 || Run Time:   28.7 | Load Time:   11.9 || F1:  99.51 | Prec:  99.03 | Rec: 100.00 || Ex/s: 182.78

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


Finished Epoch 15 || Run Time:    5.0 | Load Time:    3.9 || F1:  97.23 | Prec:  95.84 | Rec:  98.65 || Ex/s: 277.13

---------------------

Loading best model...
Training done.


tensor(97.8959, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 2


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 2 || Run Time:    5.2 | Load Time:    3.8 || F1:  98.33 | Prec:  97.35 | Rec:  99.32 || Ex/s: 274.83



tensor(98.3278, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/DBLP-GoogleScholar/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:27


Finished Epoch 1 || Run Time:   64.3 | Load Time:   24.0 || F1:  83.66 | Prec:  76.72 | Rec:  91.99 || Ex/s: 195.13

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:27


Finished Epoch 15 || Run Time:   64.1 | Load Time:   23.9 || F1:  99.64 | Prec:  99.47 | Rec:  99.81 || Ex/s: 195.79

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:19


Finished Epoch 15 || Run Time:   11.3 | Load Time:    7.8 || F1:  93.67 | Prec:  93.24 | Rec:  94.11 || Ex/s: 301.50

---------------------

Loading best model...
Training done.


tensor(93.7148, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 5 || Run Time:   11.4 | Load Time:    7.9 || F1:  94.05 | Prec:  93.61 | Rec:  94.49 || Ex/s: 296.60



tensor(94.0465, device='cuda:0')

##### Walmart-Amazon

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Structured/Walmart-Amazon/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/Walmart-Amazon/joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/Walmart-Amazon/joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Structured/Walmart-Amazon/joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:00

Computing principal components
0% [######] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 11286907
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:34


Finished Epoch 1 || Run Time:   28.2 | Load Time:    6.9 || F1:  24.18 | Prec:  55.62 | Rec:  15.45 || Ex/s: 174.93

===>  EVAL Epoch 1


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:33


Finished Epoch 15 || Run Time:   27.1 | Load Time:    6.7 || F1:  99.57 | Prec:  99.31 | Rec:  99.83 || Ex/s: 181.66

===>  EVAL Epoch 15


0% [█████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    4.7 | Load Time:    2.1 || F1:  64.36 | Prec:  66.12 | Rec:  62.69 || Ex/s: 297.19

---------------------

Loading best model...
Training done.


tensor(64.9077, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 12


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 12 || Run Time:    5.0 | Load Time:    2.2 || F1:  64.15 | Prec:  66.85 | Rec:  61.66 || Ex/s: 286.72



tensor(64.1509, device='cuda:0')

#### TEXTUAL

##### Abt-Buy

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Textual/Abt-Buy/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 7133105
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:27


Finished Epoch 1 || Run Time:   17.8 | Load Time:   10.0 || F1:  27.38 | Prec:  34.06 | Rec:  22.89 || Ex/s: 206.34

===>  EVAL Epoch 1


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:28


Finished Epoch 15 || Run Time:   18.1 | Load Time:   10.3 || F1:  98.71 | Prec:  98.08 | Rec:  99.35 || Ex/s: 202.53

===>  EVAL Epoch 15


0% [████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:06


Finished Epoch 15 || Run Time:    3.1 | Load Time:    3.1 || F1:  63.91 | Prec:  73.89 | Rec:  56.31 || Ex/s: 308.48

---------------------

Loading best model...
Training done.


tensor(67.3522, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 6


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 6 || Run Time:    3.4 | Load Time:    3.2 || F1:  65.99 | Prec:  68.59 | Rec:  63.59 || Ex/s: 288.35



tensor(65.9950, device='cuda:0')

#### DIRTY

##### DBLP-ACM

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-ACM/',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')

In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:42


Finished Epoch 1 || Run Time:   29.2 | Load Time:   13.9 || F1:  68.78 | Prec:  64.11 | Rec:  74.17 || Ex/s: 172.22

===>  EVAL Epoch 1


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:42


Finished Epoch 15 || Run Time:   28.8 | Load Time:   13.9 || F1:  99.63 | Prec:  99.25 | Rec: 100.00 || Ex/s: 173.69

===>  EVAL Epoch 15


0% [███████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:09


Finished Epoch 15 || Run Time:    5.4 | Load Time:    4.8 || F1:  95.65 | Prec:  94.70 | Rec:  96.62 || Ex/s: 242.71

---------------------

Loading best model...
Training done.


tensor(96.4286, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 5


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 5 || Run Time:    5.4 | Load Time:    4.5 || F1:  95.53 | Prec:  94.89 | Rec:  96.17 || Ex/s: 248.94



tensor(95.5257, device='cuda:0')

##### DBLP-GoogleScholar

In [ ]:
train, validation, test = dm.data.process(
    path='/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-GoogleScholar//',
    train='joined_train.csv',
    validation='joined_valid.csv',
    test='joined_test.csv')


Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-GoogleScholar//joined_train.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-GoogleScholar//joined_valid.csv"
0% [############################# ] 100% | ETA: 00:00:00
Reading and processing data from "/content/IC/datasesErros/GLow_30_1_2/Dirty/DBLP-GoogleScholar//joined_test.csv"
0% [############################# ] 100% | ETA: 00:00:00Warning : `load_model` does not return WordVectorModel or SupervisedModel any more, but a `FastText` object which is very similar.

Building vocabulary
0% [#################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01

Computing principal components
0% [#################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:08


In [ ]:
model = dm.MatchingModel(attr_summarizer='hybrid')

In [ ]:
model.run_train(
    train,
    validation,
    epochs=15,
    batch_size=32,
    best_save_path='hybrid_model.pth',
    pos_neg_ratio=3)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


* Number of trainable parameters: 9210006
===>  TRAIN Epoch 1


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward hooks on a Module that does not return a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_output. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not return a "
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:2742: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  "reduction: 'mean' divides the total loss by both the batch size and the support size."
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:33


Finished Epoch 1 || Run Time:   65.5 | Load Time:   28.2 || F1:  75.76 | Prec:  67.95 | Rec:  85.59 || Ex/s: 183.67

===>  EVAL Epoch 1


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:32


Finished Epoch 15 || Run Time:   64.9 | Load Time:   28.1 || F1:  99.72 | Prec:  99.78 | Rec:  99.66 || Ex/s: 185.18

===>  EVAL Epoch 15


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:20


Finished Epoch 15 || Run Time:   11.7 | Load Time:    9.2 || F1:  90.46 | Prec:  90.54 | Rec:  90.37 || Ex/s: 274.46

---------------------

Loading best model...
Training done.


tensor(91.4207, device='cuda:0')

In [ ]:
# Compute F1 on test set
model.run_eval(test)

===>  EVAL Epoch 4


/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:949: UserWarning: Using non-full backward hooks on a Module that does not take as input a single Tensor or a tuple of Tensors is deprecated and will be removed in future versions. This hook will be missing some of the grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using non-full backward hooks on a Module that does not take as input a "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:974: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "
/usr/local/lib/python3.7/dist-packages/torch/nn/modules/module.py:939: UserWarning: Using non-full backward h

Finished Epoch 4 || Run Time:   12.0 | Load Time:    9.3 || F1:  91.56 | Prec:  91.35 | Rec:  91.78 || Ex/s: 269.25



tensor(91.5618, device='cuda:0')

## Step 4. Apply model to new data

### Evaluating on test data
Now that we have a trained model for entity matching, we can now evaluate its accuracy on test data, to estimate the performance of the model on unlabeled data.

In [ ]:
# Compute F1 on test set
'''model.run_eval(test)'''

### Evaluating on unlabeled data

We finally apply the trained model to unlabeled data to get predictions. To do this, we need to first process the unlabeled data.

#### Processing unlabeled data

To process unlabeled data, we use `dm.data.process_unlabeled`, as shown in the code snippet below. The basic parameters for this call are as follows:

* **path (required): ** The full path to the unlabeled data file (not just the directory).
* **trained_model (required): ** The trained model. The model is aware of the configuration of the training data on which it was trained, and so `deepmatcher` reuses the same configuration for the unlabeled data.
* **ignore_columns (optional): ** Any columns in the unlabeled CSV file that you may want to ignore for the purposes of evaluation. If not specified, the columns that were ignored while processing the training set will also be ignored while processing the unlabeled data.

Note that the unlabeled CSV file must have the same schema as the train, validation and test CSVs.

In [ ]:
'''
unlabeled = dm.data.process_unlabeled(
    path='sample_data/itunes-amazon/unlabeled.csv',
    trained_model=model)
'''

#### Obtaining predictions

Next, we call the `run_prediction` method which takes a processed data set object and returns a `pandas` dataframe containing tuple pair IDs (`id` column) and the corresponding match score predictions (`match_score` column). `match_scores` are in [0, 1] and a score above 0.5 indicates a match prediction.

In [ ]:
'''
predictions = model.run_prediction(unlabeled)
predictions.head()
'''

You may optionally set the `output_attributes` parameter to also include all attributes present in the original input table. As mentioned earlier, the processed attribute values will likely look a bit different from the attribute values in the input CSV files due to modifications such as tokenization and lowercasing.

In [ ]:
#predictions = model.run_prediction(unlabeled, output_attributes=True)
#predictions.head()

You can then save these predictions to CSV and use them for downstream tasks.

In [ ]:
#predictions.to_csv('sample_data/itunes-amazon/unlabeled_predictions.csv')

#### Getting predictions on labeled data

You can also get predictions for labeled data such as validation data. To do so, you can simply call the `run_prediction` method passing the validation data as argument.

In [ ]:
#valid_predictions = model.run_prediction(validation, output_attributes=True)
#valid_predictions.head()